<a href="https://colab.research.google.com/github/penny1xu/RESTS/blob/main/python_rests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# data clean 

BathroomsFull
BathroomsHalf
BathroomsTotalDecimal
BathroomsTotalInteger
BedroomsTotal
BuildingAreaTotal
City
CloseDate
ClosePrice
CumulativeDaysOnMarket
DaysOnMarket
GarageSpaces
Latitude
ListingContractDate
ListingId
ListPrice
ListSource
Longitude
LotSizeAcres
MlsStatus
OwnerName
OwnerPhone
OwnerPhoneAlternative
ParcelNumber
ParcelNumber2
PoolYN
PostalCode
PreviousListPrice
PreviousStatus
PropertySubType
PropertyType
PublicRemarks
PurchaseContractDate
SchoolDistrict
StandardStatus
StatusChangeTimestamp
StreetDirPrefix
StreetDirSuffix
StreetName
StreetNumber
StreetNumberNumeric
StreetSuffix
SubdivisionName
USProperty_MUI
YearBuilt



In [12]:
#import package
!pip install censusgeocode
!pip install rets
import csv
import pandas as pd
import censusgeocode as cg
import numpy as np
import sys


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Data cleaning

In [26]:
import pandas as pd
import censusgeocode as cg
import numpy as np
from rets import Session
def findHouse(house_leftLa, house_rightLa, house_leftLo, house_rightLo, house_leftSqt, house_rightSqt, research, column, zip, a, time1, time2):
    basic = house_leftSqt
    count = 0
    df = pd.DataFrame()
    result = research.search(resource='Property', resource_class='Property', dmql_query='(MlsStatus = SLD), (CloseDate = 2022-01-01-2022-05-31), (YearBuilt = {}+), (YearBuilt = {}-), (Longitude = {}+), (Longitude = {}-), (Latitude = {}+), (Latitude = {}-), (LivingArea = {}+), (LivingArea = {}-), (PostalCode = {})'.format(time1, time2, house_leftLo, house_rightLo, house_leftLa, house_rightLa, house_leftSqt, house_rightSqt, zip))
    for item in result:
        if(item['PropertyType'] == 'Residential' and item['PropertySubType'] == 'Single Family Residence'):
            df = pd.concat([df, pd.DataFrame(item.values())], axis=1, ignore_index=True)
            count += 1
    if(count >= 5):
        df = df.T
        df.columns = column
        return df
    else:
        house_leftLa -= 0.001
        house_rightLa += 0.001
        house_leftLo -= 0.001
        house_rightLo += 0.001

  #Living Area
        if(a == 0):
            if(basic <= 1100):
                house_leftSqt -= 100
                house_rightSqt += 100
            elif(basic <= 2200):
                house_leftSqt -= 200
                house_rightSqt += 200
            else:
                house_leftSqt -= 300
                house_rightSqt += 300

            a += 1
        return findHouse(house_leftLa, house_rightLa, house_leftLo, house_rightLo, house_leftSqt, house_rightSqt, research, column, zip, a, time1, time2) 

def get(number, finalN, code, i, rets_client):
    pulled_value = {}
    result = rets_client.search(resource='Property', resource_class='Property', dmql_query='(StreetNumber = {}),(StreetName = {}),(PostalCode = {})'.format(number[i],finalN[i],code[i]))
    for item in result:
        pulled_value = item
    if(len(pulled_value) != 0):
        column = []
        for key in pulled_value:
            column.append(key)
        print("\nThis value exists in Website")
  # get whole row value
  #get each value
        zip = pulled_value.get('PostalCode')
        house_leftLa = float(pulled_value.get('Latitude'))
        house_rightLa = float(pulled_value.get('Latitude')) 
        house_leftLo = float(pulled_value.get('Longitude'))
        house_rightLo = float(pulled_value.get('Longitude'))
        house_leftSqt = float(pulled_value.get('LivingArea'))
        house_rightSqt = float(pulled_value.get('LivingArea'))
        time1 = int(pulled_value.get('YearBuilt'))
        time2 = int(pulled_value.get('YearBuilt'))
        if time1 < 1990:
            time1 = 0 
            time2 = 1989
        elif time1 >=1990 and time1 <2011:
            time1 = 1990
            time2 = 2010
        elif time1 >= 2011 and time1 < 2022:
            time1 = 2011
            time2 = 2021
        else:
            time1 = 2022
            time2 = 9999
        a = 0
        result1 = findHouse(house_leftLa, house_rightLa, house_leftLo, house_rightLo, house_leftSqt, house_rightSqt, rets_client, column, zip, a, time1, time2)        
        result1 = result1[['BathroomsFull','BathroomsHalf', 'BathroomsTotalDecimal', 'BathroomsTotalInteger','BedroomsTotal', 'BuildingAreaTotal', 'City', 'CloseDate', 'ClosePrice', 'CumulativeDaysOnMarket', 
   'DaysOnMarket', 'GarageSpaces', 'Latitude', 'ListingContractDate', 'ListingId', 'ListPrice', 
   'ListSource', 'LivingArea', 'Longitude', 'LotSizeAcres', 'MlsStatus', 'OwnerName', 'OwnerPhone', 'OwnerPhoneAlternative',
   'ParcelNumber', 'ParcelNumber2', 'PoolYN', 'PostalCode', 'PreviousListPrice', 'PreviousStatus', 'PropertySubType',
   'PropertyType', 'PublicRemarks', 'PurchaseContractDate', 'SchoolDistrict', 'StandardStatus', 'StatusChangeTimestamp',
   'StreetDirPrefix', 'StreetDirSuffix', 'StreetName', 'StreetNumber', 'StreetNumberNumeric', 'StreetSuffix', 'SubdivisionName',
   'USProperty_MUI', 'YearBuilt']]
        price = []
        for item in result1['ClosePrice']:
            item = float(item)
            price.append(item)
        result1['ClosePrice'] = price
        mean1 = result1['ClosePrice'].mean()
        top1 = list(result1['ClosePrice'])
        top1.sort(reverse = True)
        return result1.to_csv('{} {} {}.csv'.format(number[i],finalN[i],code[i])), mean1, top1[0:3]
    else:
        print('{},{},{} is not avaliable'.format(number[i],finalN[i],code[i]))
        result = 0
        mean = 0
        top = 0
        
        return result, mean, top

              
              
def main():
    data = pd.read_csv('test1.csv')
    code = []
    name = []
    number = []
    for item in data['地址']:
        a = item.split(',')[0]
        b = item.split(',')[2]
        numbers = a.split()[0]
        names = a.split()[1:(len(a.split()) - 1)]
        codes = b.split()[1][0:5]
        code.append(codes)
        name.append(names)
        number.append(numbers)
    finalN = []
    for item in name:
        string = ""
        for items in item:
            string += items
            string += ' '
        finalN.append(string[0:len(string) - 1])
    print(code, number, finalN)
    login_url = 'https://ntrdd.mlsmatrix.com/rets/Login.ashx' 
    username = '0671181_NID'
    password = 'Rt$tg6jx'
    rets_client = Session(login_url, username, password)
    rets_client.login()
    for i in range(len(code)):
        result, mean, top = get(number, finalN, code, i, rets_client)
        print('The mean for {}, {}, {} is {}'.format(number[i],finalN[i],code[i], mean))
        print('The top 3 for {}, {}, {} are {}'.format(number[i],finalN[i],code[i], top))
if __name__ == "__main__":
    print(main())

['76180', '75234', '76010', '75228', '75050', '75149', '76133', '76134', '75149', '75002', '75087', '75056', '76133', '75147', '75149', '75149', '75149', '75110', '76010', '75227', '75006', '75023', '76112', '76054', '76148', '75150', '75023', '75229', '76182', '75080', '75228', '76039'] ['4701', '3041', '1413', '11024', '1614', '1832', '6428', '1309', '1611', '702', '205', '5136', '7216', '1000', '703', '710', '718', '1305', '2502', '8615', '2405', '3337', '6833', '700', '5709', '3431', '3840', '3717', '7436', '200', '3034', '603'] ['Holiday Lane', 'Old North', 'Swiss', 'Delford', 'Capetown', 'Buena Vista', 'Woodway', 'Milmo', 'Cascade', 'Spring Brook', 'Summit Ridge', 'Sherman', 'Ridge Road', 'S Andrea', 'Sara', 'Sara', 'Sara', 'Tammy', 'Reever', 'Barclay', 'Briarwood', 'Whiffletree', 'Hightower', 'Highland Crest', 'Shipp', 'Caracas', 'Silverstone', 'Duchess', 'Chelmsford', 'Murray', 'Housley', 'Donley']
4701,Holiday Lane,76180 is not avaliable
The mean for 4701, Holiday Lane, 76180 

KeyboardInterrupt: ignored